## Merge SAP with TAX data

Combine data from SAP and Manuals.
SAP data contains the structure. All stands are listed in the right sequence.
Manuals contain the changes to be made.

In [1]:
import pandas as pd
import numpy as np

In [2]:
class CreateInput(object):

    def __init__(self, wo_data, tax_data):
        '''
        input:
            wo_data - DataFrame with the WO structure (raw data export from SAP)
            tax_data - DataFrame with the updated tax data
        '''
        
        # prepare wo_data
        self.wo_data = self.prepare_wo_data(wo_data)
        # filter data
        self.wo_struct = self.wo_data.loc[self.wo_data['Best.-Schicht']==0,['Waldort', 'WE-Typ']]
        # prepare tax_data
        self.tax_data = self.prepare_tax_data(tax_data)
        self.clean_prefix()
        
        self.bas = ['FI', 'TA', 'LA', 'KI', 'SK', 'ZI', 'PM', 'DG', 'KW', 'SF', 'FO', 'TH', 'FB',\
                    'AC', 'AG', 'AZ', 'EB', 'FZ', 'GK', 'HT', 'JL', 'CJ', 'KK', 'KO', 'AN', 'AB',\
                    'CH', 'PU', 'SN', 'BU', 'EI', 'HB', 'AH', 'SA', 'FA', 'EA', 'ES', 'UL', 'QP',\
                    'QR', 'EZ', 'RE', 'FE', 'ER', 'GE', 'AV', 'KB', 'TK', 'WO', 'SG', 'NU', 'JN',\
                    'LI', 'LS', 'LW', 'BI', 'PO', 'AS', 'WP', 'SP', 'HP', 'WD', 'SW', 'EK', 'RK',\
                    'EE', 'EL', 'ME', 'RO', 'TB', 'GB', 'ST', 'SL', 'BL']
        
        
    def prepare_wo_data(self, wo_data):
        ###   wo_data   ###
        
        # sort WOs
        wo_data.sort_values(by=['Abteilung', 'WE-Typ', 'Unterabteil.', 'Teilfl.'], ascending=[True,False,True,True], inplace=True)
        
        # construct WO-name
        wo_data['Waldort'] = wo_data['Abteilung'].astype(str).str.pad(width=3, side='left', fillchar='0') + wo_data['Unterabteil.'] + wo_data['Teilfl.'].astype(str)
        
        # add bz_row information
        wo_data = self.add_bz_rows(wo_data)
        
        # add ba_row information
        wo_data = self.add_ba_rows(wo_data)
        
        # filter just unique WOs
        wo_data.loc[wo_data['Best.-Schicht']==0]
        
        return wo_data
    
    
    def prepare_tax_data(self, tax_data):
        ###   tax_data   ###
        
        # get number of max measures (Nutzungen) per unit (Bestand)
        self.nr_ma = int(tax_data.columns[-2][5:])

        # clean WO-name
        tax_data['Waldort'] = tax_data['WO'].str.replace(" ", "")
    
        return tax_data
    
    
    def clean_prefix(self):
        for col, i in [('U', 2), ('W', 2), ('STOE', 5), ('VTYP', 5), ('VB', 3), ('UELH', 5), ('UENH', 5), ('BZ', 3)]:
            if self.tax_data.loc[~pd.isna(self.tax_data[col]),col].size > 0:
                self.tax_data.loc[~pd.isna(self.tax_data[col]),col] = self.tax_data.loc[~pd.isna(self.tax_data[col]),col].str[i:]
                
    
    
    #############################
    ###   create data files   ###
    #############################
    
    
    def create_text(self):
        # merge data
        merge_data = pd.merge(self.wo_struct, self.tax_data[['Waldort','ST', 'BE', 'MA']], how='left', on='Waldort')
        merge_data.to_csv('autax/text.csv', index=False)
        print('*** created autax *text* input file in "autax/text.csv" ***')
        
        
    def create_bz(self):
        
        # create empty dataframe
        wo_bz = self.wo_struct.copy()
        wo_bz[self.bas] = 0

        # filter tax data for BZ
        tax_bz = self.tax_data.loc[~pd.isna(self.tax_data['BZ']),['Waldort', 'BZ']].copy()
        
        for idx, inst in tax_bz.iterrows():
            nums, bas = self.check_bz(inst['BZ'])
            
            if nums:
                wo_bz.loc[wo_bz['Waldort']==inst['Waldort'], bas] = nums
                
        wo_bz.to_csv('autax/bz.csv', index=False)
        print('*** created autax *bz* input file in "autax/bz.csv" ***')
        
        
    def create_nutz(self):

        for i in range(self.nr_ma):
            # create column names
            col = ['Waldort', 'S_{}'.format(i), 'MA_{}'.format(i), 'Fl_{}'.format(i), 'LH_{}'.format(i),\
                       'NH_{}'.format(i), 'Dring_{}'.format(i), 'Zeit_{}'.format(i), 'Bew_{}'.format(i),\
                       'Sch_{}'.format(i), 'Rueck_{}'.format(i), 'Text_{}'.format(i)]

            # merge data
            merge_data = pd.merge(self.wo_data.loc[self.wo_data['Best.-Schicht']==0,['Waldort', 'WE-Typ', 'bz_rows', 'ba_rows']],\
                                  self.tax_data[col], how='left', on='Waldort')

            # rename columns
            merge_data.columns = ['Waldort', 'WE-Typ', 'bz_rows', 'ba_rows', 'S', 'MA', 'Fl', 'LH', 'NH', 'Dring', 'Zeit', 'Bew', 'Sch', 'Rueck', 'Text']

            # convert , to .
            merge_data['Fl'] = merge_data['Fl'].str.replace(",", ".")

            # adjust data type
            merge_data = merge_data.fillna(0)
            merge_data['bz_rows'] = merge_data['bz_rows'].astype(int)
            merge_data['ba_rows'] = merge_data['ba_rows'].astype(int)
            merge_data['S'] = merge_data['S'].astype(int)
            merge_data['Fl'] = merge_data['Fl'].astype(float)
            merge_data['LH'] = merge_data['LH'].astype(int)
            merge_data['NH'] = merge_data['NH'].astype(int)
            merge_data['Dring'] = merge_data['Dring'].astype(int)
            merge_data['Zeit'] = merge_data['Zeit'].astype(int)
            merge_data['Bew'] = merge_data['Bew'].astype(int)
            merge_data['Sch'] = merge_data['Sch'].astype(int)
            merge_data['Rueck'] = merge_data['Rueck'].astype(int)
            
            merge_data.to_csv('autax/nutz_{}.csv'.format(i) , index=False)
            print('*** created autax *nutz* input file in "autax/nutz_{}.csv" ***'.format(i))
            
            
    def create_wo(self):
        self.decode_W()
        merge_data = pd.merge(self.wo_struct, self.tax_data[['Waldort', 'U', 'I_A', 'W_S', 'SW', 'STOE', 'VTYP', 'UELH', 'UENH', 'VB']], how='left', on='Waldort')
        merge_data = merge_data.fillna('0')
        merge_data.to_csv('autax/wo.csv', index=False)
        print('*** created autax *wo* input file in "autax/wo.csv" ***')
            
    
    def decode_W(self):
        self.tax_data.loc[self.tax_data['W'] == 'WI', ['W_S', 'SW', 'I_A']] = ['W', '0', 'I']
        self.tax_data.loc[self.tax_data['W'] == 'WA', ['W_S', 'SW', 'I_A']] = ['W', '0', 'A']
        self.tax_data.loc[self.tax_data['W'] == 'SSI', ['W_S', 'SW', 'I_A']] = ['S', 'S', 'I']
        self.tax_data.loc[self.tax_data['W'] == 'SSA', ['W_S', 'SW', 'I_A']] = ['S', 'S', 'A']
        self.tax_data.loc[self.tax_data['W'] == 'OSI', ['W_S', 'SW', 'I_A']] = ['O', 'S', 'I']
        self.tax_data.loc[self.tax_data['W'] == 'OSA', ['W_S', 'SW', 'I_A']] = ['O', 'S', 'A']
        self.tax_data.loc[self.tax_data['W'] == 'BSI', ['W_S', 'SW', 'I_A']] = ['B', 'S', 'I']
        self.tax_data.loc[self.tax_data['W'] == 'BSA', ['W_S', 'SW', 'I_A']] = ['B', 'S', 'A']


    def check_bz(self, bz_str):
        '''
        Checks Bestockungsziel (bz) for validity and returns the splited data.
        '''
        nums = []
        bas = []
        
        # split sting into bz instances
        bz_split = bz_str.split()

        for bz_inst in bz_split:
            try:
                if bz_inst[-2:] in self.bas:
                    nums.append(int(bz_inst[:-2]))
                    bas.append(bz_inst[-2:])
                else:
                    print('Not a valid tree species {}'.format(bz_inst))

            except:
                print('Input not valid')

        if sum(nums) != 10:
            print('BZ does not add up to 10')

        return nums, bas
    

    def add_bz_rows(self, wo_data):
        # filter data just wo (forest stands) instances
        try:
            wo_data['bz_rows'] = wo_data.loc[wo_data['Best.-Schicht']==0, 'Bestockungsziel'].str.count(' ') + 1
        except:
            wo_data['bz_rows'] = 0
        return wo_data
        
    def add_ba_rows(self, wo_data):
        # filter data just ba (tree species) instances
        ba_data = wo_data.loc[wo_data['Best.-Schicht.1'] != 0, ['Waldort', 'Status']].copy()
        # get number of ba (tree species) instances per wo (forest stand)
        ba_data = ba_data.groupby(['Waldort']).count()
        # rename column
        ba_data.columns = ['ba_rows']
        # set Waldort as column
        ba_data = ba_data.reset_index()
        # merge data
        wo_data = pd.merge(wo_data, ba_data, how='left', on='Waldort')
        return wo_data

Read data from SAP and TAX and create CreateInput instance

In [3]:
path = '/home/philipp/Data/obf_autman/SAP/raw_1420_02_20211122.XLS'
wo_struct = pd.read_csv(path, sep='\t', encoding = "ISO-8859-1", decimal=',', error_bad_lines=False)

path = 'clean/Manuale_Dross.xlsx'
tax_data = pd.read_excel(path)

ci = CreateInput(wo_struct, tax_data)

/home/philipp/miniconda3/envs/autman/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (29,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Create input data for autax fillin

In [4]:
ci.create_wo()
ci.create_text()
ci.create_bz()
ci.create_nutz()

*** created autax *wo* input file in "autax/wo.csv" ***
*** created autax *text* input file in "autax/text.csv" ***
*** created autax *bz* input file in "autax/bz.csv" ***
*** created autax *nutz* input file in "autax/nutz_0.csv" ***
*** created autax *nutz* input file in "autax/nutz_1.csv" ***


In [3]:
path_tax = '/home/philipp/Code/python/obf_autman/autax/text.csv'
path_sap = '/home/philipp/Code/python/obf_autman/autax/text_172_02_.csv'

text_tax = pd.read_csv(path_tax)
text_sap = pd.read_csv(path_sap)

In [4]:
text_tax

,Waldort,WE-Typ,ST,BE,MA
0,053A1,WO,"ST im N steiler sowie tiefgründiger, örtl Be...","BE im S mehr KI, im N mehr FI, im NO mehr LÄ...","MA LI: TA fördern, FI und geringkronige LA/K..."
1,053A2,WO,ST STOE 71/72,"BE BI, ein KI, FI-reicher im W-Spitz, FI-Str...","MA geastet, EDF 2005, DF 2021, ende Dez. Kon..."
2,053A3,WO,ST STOE 71/72,BE mehr LH im S - dort auch zlr BI,"MA geastet, EDF 2005, DF 2021, weitere DF en..."
3,053A4,WO,ST STOE 71/72,"BE bis 5 J älter, LA teils bereits kleine Krone","MA tlw EDF 2005, DF 2021, weitere DF nächste..."
4,053B1,WO,"ST im S STOE 71, dicht","BE FI grtls in N-Hälfte - dort auch eiz KO, ...",MA LI/PL TA unbedingt erhalten
...,...,...,...,...,...
2932,752E2,WO,NaN,"BE LH jünger, LÄ tls bedrängt teils bereits ...","MA DE 2017, DF LA fördern, HB zurückdrängen"
2933,752E3,WO,NaN,"BE Streifenweise angelegt, N-Tl etw älter, i...","MA DE 2009/17, ZV Käfer 2018-2021; DF"
2934,752F1,WO,NaN,NaN,NaN
2935,752F2,WO,NaN,NaN,NaN


In [5]:
data_temp = data.loc[~pd.isna(data[wo_type]),['WO', wo_type]].copy()

NameError: name 'data' is not defined

In [7]:
# add text from 
for wo_type in ['ST', 'BE', 'MA']:
    text_tax.loc[pd.isna(text_tax[wo_type]), wo_type] = text_sap.loc[pd.isna(text_tax[wo_type]), wo_type]

In [8]:
text_tax

,Waldort,WE-Typ,ST,BE,MA
0,053A1,WO,"ST im N steiler sowie tiefgründiger, örtl Be...","BE im S mehr KI, im N mehr FI, im NO mehr LÄ...","MA LI: TA fördern, FI und geringkronige LA/K..."
1,053A2,WO,ST STOE 71/72,"BE BI, ein KI, FI-reicher im W-Spitz, FI-Str...","MA geastet, EDF 2005, DF 2021, ende Dez. Kon..."
2,053A3,WO,ST STOE 71/72,BE mehr LH im S - dort auch zlr BI,"MA geastet, EDF 2005, DF 2021, weitere DF en..."
3,053A4,WO,ST STOE 71/72,"BE bis 5 J älter, LA teils bereits kleine Krone","MA tlw EDF 2005, DF 2021, weitere DF nächste..."
4,053B1,WO,"ST im S STOE 71, dicht","BE FI grtls in N-Hälfte - dort auch eiz KO, ...",MA LI/PL TA unbedingt erhalten
...,...,...,...,...,...
2932,752E2,WO,"ST Standortseinheit 81 72,","BE LH jünger, LÄ tls bedrängt teils bereits ...","MA DE 2017, DF LA fördern, HB zurückdrängen"
2933,752E3,WO,"ST Standortseinheit 81 72, im Osten etwas st...","BE Streifenweise angelegt, N-Tl etw älter, i...","MA DE 2009/17, ZV Käfer 2018-2021; DF"
2934,752F1,WO,"ST oben Felsen aus Gneis und Marmor, Standor...",BE dichteres Altholz im Süden dort vor allem...,0
2935,752F2,WO,ST Bittescher Gneis silikatisch und Marmor k...,"BE ältere Schicht 20 J, ST, vor allem Hartri...",0


In [9]:
text_tax.to_csv('autax/autax_text.csv', index=False)